In [1]:
from __future__ import print_function
import sys
import IPython
print('IPython:', IPython.__version__)
import numpy
print('numpy:', numpy.__version__)
import pandas
print('pandas:', pandas.__version__)
import sklearn
print('scikit-learn:', sklearn.__version__)
print('Python version:', sys.version)

IPython: 7.34.0
numpy: 1.25.2
pandas: 1.5.3
scikit-learn: 1.2.2
Python version: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/DATA QUALITY MANAGEMENT')

In [4]:
%cd /content/drive/My Drive/Colab Notebooks/DATA QUALITY MANAGEMENT

/content/drive/My Drive/Colab Notebooks/DATA QUALITY MANAGEMENT


In [5]:
import pandas as pd
import numpy as np
import random
import string
import re

In [6]:
def generate_valid_name():
    valid_name = ''

    # Generate first name
    first_name = ''.join(random.choices(string.ascii_uppercase, k=1)) + ''.join(random.choices(string.ascii_lowercase, k=random.randint(2, 7)))
    valid_name += first_name

    # Generate middle name
    if random.random() < 0.5:
        middle_name = ''.join(random.choices(string.ascii_uppercase, k=1)) + ''.join(random.choices(string.ascii_lowercase, k=random.randint(2, 7)))
        valid_name += ' ' + middle_name

    # Generate last name
    last_name = ''.join(random.choices(string.ascii_uppercase, k=1)) + ''.join(random.choices(string.ascii_lowercase, k=random.randint(2, 7)))
    valid_name += ' ' + last_name

    return valid_name

def generate_invalid_name():
    invalid_chars = string.punctuation + string.digits
    invalid_name = ''

    # Generate first name
    first_name = ''.join(random.choices(string.ascii_lowercase, k=random.randint(3, 8)))
    if random.random() < 0.5:
        first_name = ''.join(random.choices(invalid_chars, k=random.randint(1, 3))) + first_name
    invalid_name += first_name

    # Generate middle name
    if random.random() < 0.5:
        middle_name = ''.join(random.choices(string.ascii_lowercase, k=random.randint(3, 8)))
        if random.random() < 0.5:
            middle_name = ''.join(random.choices(invalid_chars, k=random.randint(1, 3))) + middle_name
        invalid_name += ' ' + middle_name

    # Generate last name
    last_name = ''.join(random.choices(string.ascii_lowercase, k=random.randint(3, 8)))
    if random.random() < 0.5:
        last_name = ''.join(random.choices(invalid_chars, k=random.randint(1, 3))) + last_name
    invalid_name += ' ' + last_name

    # Add double space
    if random.random() < 0.3:
        invalid_name = invalid_name.replace(' ', '  ')

    return invalid_name

def validate_name(text):
    pattern = r'^[a-zA-Z]+(?: [a-zA-Z]+)*$'
    if re.match(pattern, text):
        return 1
    else:
        return 0

def make_teks_proper(text):
    # Replace special character and numbers in the middle of string with space
    cleaned_string = re.sub(r'[^A-Za-z\s]+', ' ', text)
    # Remove non alphabetic character except space
    cleaned_string = re.sub(r'\s{2,}', r' ', cleaned_string)
    return cleaned_string.strip()  # Remove space in the first and last string

def upperCaseCheck(name):
    pattern = r'^[A-Z][a-z\s]*$'
    return re.match(pattern, name) is not None


In [7]:
data  = []
initial_label = []
check_type = []

for _ in range(700):
  data.append(generate_valid_name())
  initial_label.append(1)
  check_type.append('name')

for _ in range(700):
  data.append(generate_invalid_name())
  initial_label.append(1)
  check_type.append('name')

In [8]:
dataset = pd.DataFrame({"text" : data, 'initial_label' : initial_label, 'data_type': check_type})
dataset.to_csv('dataset/name_dataset.csv')
dataset

,text,initial_label,data_type
0,Yik Ieynkvdk,1,name
1,Dba Gal,1,name
2,Vweq Aapyz,1,name
3,Sei Xkvtx,1,name
4,Vxc Anjevnne Dxijqtcg,1,name
...,...,...,...
1395,mtxkg qhubxw,1,name
1396,inhqhh bnmsjd [hlgjta,1,name
1397,gniurpkj lgb,1,name
1398,!3lkqlqo /}zpplmkz,1,name


In [9]:
dataset['label'] = dataset['text'].apply(lambda x: validate_name(x))
dataset['clean'] = dataset.apply(lambda x: make_teks_proper(str(x['text'])) if x['label'] == False else x['text'], axis=1)
dataset['check'] = dataset['clean'].apply(lambda x: validate_name(x))

dataset[['text','label','clean','check']]

,text,label,clean,check
0,Yik Ieynkvdk,1,Yik Ieynkvdk,1
1,Dba Gal,1,Dba Gal,1
2,Vweq Aapyz,1,Vweq Aapyz,1
3,Sei Xkvtx,1,Sei Xkvtx,1
4,Vxc Anjevnne Dxijqtcg,1,Vxc Anjevnne Dxijqtcg,1
...,...,...,...,...
1395,mtxkg qhubxw,0,mtxkg qhubxw,1
1396,inhqhh bnmsjd [hlgjta,0,inhqhh bnmsjd hlgjta,1
1397,gniurpkj lgb,0,gniurpkj lgb,1
1398,!3lkqlqo /}zpplmkz,0,lkqlqo zpplmkz,1


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

data = dataset
# Split Feature (X) and label (Y)
X = data['text'].values
Y = data['label'].values

# Splitting data Training and Testing
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

X

array(['Yik Ieynkvdk', 'Dba Gal', 'Vweq Aapyz', ..., 'gniurpkj  lgb',
       '!3lkqlqo /}zpplmkz', '4~+kldbc kyxlajs'], dtype=object)

In [16]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from transformers import BertTokenizer, BertModel
import torch

email_addresses = X
labels = Y

# Load pre-trained tokenizer dan model (BERT)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [18]:
# Tokenize email addresses
tokenized_emails = [tokenizer.encode(email, add_special_tokens=True) for email in email_addresses]

# Padding tokenized emails
max_len = max([len(email) for email in tokenized_emails])
padded_emails = np.array([email + [0]*(max_len-len(email)) for email in tokenized_emails])

# Convert tokenized emails to tensors
input_ids = torch.tensor(padded_emails)

# Create attention masks
attention_masks = torch.tensor([[1]*len(email) + [0]*(max_len-len(email)) for email in tokenized_emails])

# Extract features from BERT model
features = []
with torch.no_grad():
    for input_id, attention_mask in zip(input_ids, attention_masks):
        input_id = input_id.unsqueeze(0)  # Add batch dimension
        attention_mask = attention_mask.unsqueeze(0)  # Add batch dimension
        outputs = model(input_id, attention_mask=attention_mask)
        last_hidden_states = outputs.last_hidden_state
        email_features = last_hidden_states.mean(dim=1).squeeze().numpy()  # Take average embedding for each token
        features.append(email_features)



In [20]:

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Convert data to tensors
X_train = torch.tensor(np.array(X_train)).float()
X_test = torch.tensor(np.array(X_test)).float()

In [22]:
# Train classification model
classifier = LogisticRegression(max_iter=1000)  # Increase max_iter
classifier.fit(X_train, y_train)

# Predict labels for test data
predictions = classifier.predict(X_test)

# Evaluate model
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

# Print classification report
report = classification_report(y_test, predictions)
print("Classification Report:\n", report)

Accuracy: 0.9678571428571429
Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.94      0.96       124
           1       0.96      0.99      0.97       156

    accuracy                           0.97       280
   macro avg       0.97      0.97      0.97       280
weighted avg       0.97      0.97      0.97       280



In [23]:
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin, BaseEstimator
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Custom transformer to extract BERT features
class BertFeatureExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, tokenizer, model):
        self.tokenizer = tokenizer
        self.model = model

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        tokenized_emails = [self.tokenizer.encode(email, add_special_tokens=True) for email in X]
        max_len = max([len(email) for email in tokenized_emails])
        padded_emails = np.array([email + [0]*(max_len-len(email)) for email in tokenized_emails])
        input_ids = torch.tensor(padded_emails)
        attention_masks = torch.tensor([[1]*len(email) + [0]*(max_len-len(email)) for email in tokenized_emails])

        features = []
        with torch.no_grad():
            for input_id, attention_mask in zip(input_ids, attention_masks):
                input_id = input_id.unsqueeze(0)  # Add batch dimension
                attention_mask = attention_mask.unsqueeze(0)  # Add batch dimension
                outputs = self.model(input_id, attention_mask=attention_mask)
                last_hidden_states = outputs.last_hidden_state
                email_features = last_hidden_states.mean(dim=1).squeeze().numpy()  # Take average embedding for each token
                features.append(email_features)
        return features

# Define BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Define pipeline
pipeline = Pipeline([
    ('bert_feature_extractor', BertFeatureExtractor(tokenizer, model)),
    ('logistic_regression', LogisticRegression(max_iter=1000))  # Increase max_iter
])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(email_addresses, labels, test_size=0.2, random_state=42)

# Train pipeline
pipeline.fit(X_train, y_train)

# Predict labels for test data
predictions = pipeline.predict(X_test)

# Evaluate pipeline
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)


Accuracy: 0.9678571428571429


In [24]:
pipeline

Pipeline(steps=[('bert_feature_extractor',
                 BertFeatureExtractor(model=BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder)...
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
})),
                ('logistic_regression', LogisticRegression(max_iter=1000))])

In [34]:
# Sample new email addresses for prediction
new_email_addresses = [
    "John Rush",
    "Raiso Loro",
    "Hirata 1st onest",
]

# Make predictions using the pipeline model
predictions = pipeline.predict(new_email_addresses)

# Print predictions
for email, prediction in zip(new_email_addresses, predictions):
    print(f"Email: {email}, Predicted Label: {'Valid' if prediction == 1 else 'Invalid'} {prediction}")


Email: John Rush, Predicted Label: Valid 1
Email: Raiso Loro, Predicted Label: Invalid 0
Email: Hirata 1st onest, Predicted Label: Invalid 0


In [ ]:
features